# Logistic Regression

Today we will start working on our second Machine Learning model: Logistic Regression.

# Table of contents
[1. Logistic Regression](#1.-Linear-Regression)
- [1.0 Logistic Regression Example](#1.0-Logistic-Regression-Example:-Can-you-take-me-to-Mars?)
- [1.1 Using logistic regression with the Fragile Families data](#1.1-Using-logistic-regression-with-the-Fragile-Families-data)
- [1.2 Plotting the data](#1.2-Plotting-the-data)
- [1.3 Splitting data into train and test sets](#1.3-Splitting-data-into-train-and-test-sets)
- [1.4 Running the logistic regression algorithm](#1.4-Running-the-logistic-regression-algorithm)
- [1.5 Evaluating the logistic regression model](#1.5-Evaluating-the-logistic-regression-model)


[2. Multiple Logistic Regression](#2.-Multiple-Linear-Regression)

# 1. Logistic Regression

Logistic regression models the relationship between a binary outcome $Y$ that can be either 1 or 0 and a predictor $X$. The main difference between logistic regression and linear and multiple linear regression is that the resulting model is not linear (it is not a line but a curve) and that it does not predict the value of the outcome $Y$ but instead it predicts the probability that $Y$ is equal to 1.

As in linear regression, we plot the data on a scatter plot and then compute the best-fit curve through the data points using the following equation which takes into account that $Y$ is a binary variable:

$$ \ln \bigg( \frac{p(Y)}{1-p(Y)} \bigg) = c_0 + c_1 X$$

where $p(Y)$ is the probability that $Y=1, $ $c_0$ is the regression coefficient associated with the probability that $Y=1$ when $X=0$, and $c_1$ is the regression coefficient that tells us how much the probability that $Y=1$ changes when the predictor $X$ is not zero.

The best-fit curve provides a nonlinear model which allows us to make predictions about the probability that $Y=1$ or $Y=0$ given the value of $X$.

In this section we will build a logistic regression model to predict layoff. For the predictor $X$, we will use information about whether the mother had "done regular work for pay last week" in wave 5 (when the child was ~9 years old) (feature m5i4).

In [ ]:
# First, we load the libraries we will need for this notebook
%matplotlib inline 
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import pandas as pd
import sys
import sklearn # this library provides functions for performing machine learning on data
import ff_functions as fff # this is the library we are providing for you to perform common operations on data

# 1.0 Logistic Regression Example: Can you take me to Mars?

Before we apply logistic regression to the Fragile Families data, let's look at a quick example.

Imagine we want to predict whether a student in Sebastian's plasma propulsion lab can build a plasma thruster to go to Mars. We want to base our prediction on how many years the student has been working in the lab.

Let's make some fake data for this.

In [ ]:
magic_num_years = 5
can_go2mars = []
years_in_lab = 10*np.random.random(size=(1000,1))
for student in years_in_lab:
    if student < magic_num_years:
        if np.random.random() > 0.99:
            can_go2mars.append(1)    
        else:
            can_go2mars.append(0)
    else:
        if np.random.random() < 0.99:
            can_go2mars.append(1)    
        else:
            can_go2mars.append(0)  
            
df_fake = pd.DataFrame(years_in_lab)
df_fake['can_go2mars'] = can_go2mars
df_fake.columns = ['num years','can go 2 Mars']

df_fake.head()

### Plot fake data

In [ ]:
# How many students can go to mars and how many can't?
yes = df_fake[df_fake['can go 2 Mars']==1].shape[0]
no = df_fake[df_fake['can go 2 Mars']==0].shape[0]

fff.one_bar_chart([yes,no],label="Number of students that can go to Mars",bar_labels=["Yes","No"])

In [ ]:
fff.scatterplot(df_fake['num years'],df_fake['can go 2 Mars'],
                xlabel="Num Years",ylabel="Can go to Mars",title="Fake Data")

### Let's split data into training and testing sets

In [ ]:
# First, let's split our data into train and test sets
from sklearn.model_selection import train_test_split

X = df_fake['num years'] # predictor
y = df_fake['can go 2 Mars'] # outcome 
# The following line of code randomly splits both the outcome data and the predictor data into 
# a training and a testing set. The test_size argument is a number between 0 and 1 that tells the 
# function the percentage of data that should go into the test set.
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.3, random_state=100)

print(df_fake.shape)
print(train_X.shape)
print(test_X.shape)
print(train_y.shape)
print(test_y.shape)

In [ ]:
# Transform Pandas Series into numpy arrays
train_X = fff.pandas_to_2d_numpy(train_X)
train_y = fff.pandas_to_2d_numpy(train_y)

test_X = fff.pandas_to_2d_numpy(test_X)
test_y = fff.pandas_to_2d_numpy(test_y)

### Construct and Train Linear Regression Model

In [ ]:
from sklearn import linear_model

# intialize the model
log_reg = linear_model.LogisticRegression()

# fit the model ON THE TRAINING DATA
log_reg.fit(train_X, train_y.ravel())

# read the resulting model's slope and intercept values
slope = log_reg.coef_[0][0]
intercept = log_reg.intercept_[0]

# print the linear regression model coefficients
print('slope = {}'.format(slope))
print('y-intercept = {}'.format(intercept))

### Visualize Logistic Regression Model

In [ ]:
def scatterplot_with_model(xdata, ydata, reg, xlabel="", ylabel="", title=""):
    fig = plt.figure(figsize=(7,5))
    plt.scatter(xdata, ydata)
    x = np.linspace(min(xdata)-0.5,max(xdata)+0.5, 100)
    x = x.reshape(x.shape[0],1)
    loss = reg.predict(x)
    plt.plot(x, loss, color='red', linewidth=3)
    plt.xlim(min(xdata)-0.5,max(xdata)+0.5)
    plt.ylim(min(ydata)-0.5,max(ydata)+0.5)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    
scatterplot_with_model(test_X, test_y, log_reg,
                          xlabel = "Average age", ylabel = "HH Income",\
                          title="Testing Data")

### Testing the model

#### Classification Accuracy

We will use classification accuracy to quantify the performance of our model. Classification accuracy is the number of correct predictions made as a ratio of all predictions made.

This is the most common evaluation metric for classification problems, it is also the most misused. It is really only suitable when there are an equal number of observations in each class (which is rarely the case) and that all predictions and prediction errors are equally important, which is often not the case.

In [ ]:
# Use score method to get accuracy of model
score = log_reg.score(test_X, test_y)
print(score)

#### ROC Curve

In Data Science, evaluating model performance is very important and the most commonly used performance metric is the classification accuracy. However, when dealing with datasets with heavy class imbalance, a classification score does not make much sense. Instead, Receiver Operating Characteristic or ROC curves offer a better alternative. ROC is a plot of signal (True Positive Rate) against noise (False Positive Rate). The model performance is determined by looking at the area under the ROC curve (or AUC). The best possible AUC is 1 while the worst is 0.5 (the 45 degrees random line). Any value less than 0.5 means we can simply do the exact opposite of what the model recommends to get the value back above 0.5.

Check out [this website](https://towardsdatascience.com/receiver-operating-characteristic-curves-demystified-in-python-bd531a4364d0) for more info on ROC curves.

In [ ]:
from sklearn.metrics import roc_curve, auc
y_pred = log_reg.predict(test_X)
fpr, tpr, _ = roc_curve(test_y, y_pred)
roc_auc = auc(fpr, tpr)
print(roc_auc)

### Visualizing the ROC Curve

In [ ]:
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

# 1.1 Using logistic regression with the Fragile Families data

## Predicting child's low birth weight using father's engagement as predictor

In this section we will build a logistic regression model to predict child's low birth weight. As a predictor, we will use an engineered feature, which we constructed on day 3, following a paper published in March 2018: "Father Early Engagement Behaviors and Infant Low Birth Weight" by Lee et al.

The paper combined the following three features into a single "father involvement" variable.

* "f1b16": During the baby mother's preg, did you give her money or buy things for the baby/ies?
* "f1b17": Did you help in other ways, like providing transportation / doing chores?
* "f1a2": Were you present at the birth?

They then used the engineered feature to predict the following **binary** variable:
* "cm1lbw": Constructed variable - Low Birth Weight

We start by loading the Fragile Families data.

In [ ]:
# We read in the Fragile Families data as in previous notebooks
background = "../ff_data/background.csv"
data_frame = pd.read_csv(background, low_memory=False)
# We set the index of the DataFrame to match the challengeID
data_frame = data_frame.set_index('challengeID')

## Problem 0: Subselecting variables

In [ ]:
# Pick low birth weight (cm1lbw), whether the father gave money to the mother during pregnancy (f1b16), 
# whether the father helped in other ways (f1b17), and whether the father was present at birth (f1a2)
df = fff.pick_ff_variables(#INSERT FUNCTION INPUTS HERE)

# Let's change the names of the columns so it's easier for us to work with the data
df.columns = ["low birth weight","gave money", "other help", "present at birth"]

df.head()

## Problem 1:

Create a new feature called "father engagement" by taking the average of the following three columns:

- gave money
- other help
- present at birth

In [ ]:
### Generating an engineered feature:
df["father engagement"] = # COMPUTE FATHER ENGAGEMENT HERE
df.head()

### Discuss with your partner:
* Why would the combination of the three features be a better predictor than using the father's providing money only? What other weighings of the three variables would you consider and why?

### Answer:

# 1.2 Plotting the data
Let's plot the data using the scatterplot function we learned.

In [ ]:
fff.scatterplot(df['father engagement'], 
                df['low birth weight'], 
                xlabel = "Father's engagement",
                ylabel = "Low birth weight")

In [ ]:
lbw_no = df[df['low birth weight']==0].shape[0]
lbw_yes = df[df['low birth weight']==1].shape[0]
print("Number of datapoints without low birth weight: ",lbw_no)
print("Number of datapoints with low birth weight: ",lbw_yes)

## Discuss: 

Do you think the fact that there a lot more "no" than "yes" for low birth weight makes it easier or harder to predict?
Why?

# 1.3 Splitting data into train and test sets

## Problem 2:  Fill in the blanks

In [ ]:
# First, let's split our data into train and test sets
from sklearn.model_selection import train_test_split

X = # FILL HERE WITH PREDICTOR DATA # predictor
y = # FILL HERE WITH OUTCOME DATA # outcome 
# The following line of code randomly splits both the outcome data and the predictor data into 
# a training and a testing set. The test_size argument is a number between 0 and 1 that tells the 
# function the percentage of data that should go into the test set.
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.3, random_state=100)

print(df.shape)
print(train_X.shape)
print(test_X.shape)
print(train_y.shape)
print(test_y.shape)

In [ ]:
# Transform Pandas Series to numpy arrays
train_X = fff.pandas_to_2d_numpy(train_X)
train_y = fff.pandas_to_2d_numpy(train_y)

test_X = fff.pandas_to_2d_numpy(test_X)
test_y = fff.pandas_to_2d_numpy(test_y)

# 1.4 Running the logistic regression algorithm
**Let's do linear regression with `sklearn.linear_model.LogisticRegression`** You can learn about this function at 
http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [ ]:
from sklearn import linear_model

# intialize the model
log_reg = linear_model.LogisticRegression()

# fit the model ON THE TRAINING DATA
log_reg.fit(train_X, train_y.ravel())

# read the resulting model's slope and intercept values
slope = log_reg.coef_[0][0]
intercept = log_reg.intercept_[0]

# print the linear regression model coefficients
print('slope = {}'.format(slope))
print('y-intercept = {}'.format(intercept))

## Problem 3: Plot the resulting model

In [ ]:
scatterplot_with_model(# FILL IN HERE WITH FUNCTION INPUTS)

## Discuss: 

Why does our model look like a straight line? And what does this mean in terms of the model's predictions?

# 1.5 Evaluating the logistic regression model

We will use classification accuracy to quantify the performance of our model. Classification accuracy is the number of correct predictions made as a ratio of all predictions made.

This is the most common evaluation metric for classification problems, it is also the most misused. It is really only suitable when there are an equal number of observations in each class (which is rarely the case) and that all predictions and prediction errors are equally important, which is often not the case.

## Problem 4: compute classification accuracy

In [ ]:
# Use score method to get accuracy of model
score = # FILL IN WITH SCORE 
print(score)

## Discuss: 

- Do you think using classification accuracy is the right thing to do in our case?

- How can we fix this?

# ROC Curve

In Data Science, evaluating model performance is very important and the most commonly used performance metric is the classification accuracy. However, when dealing with datasets with heavy class imbalance, a classification score does not make much sense. Instead, Receiver Operating Characteristic or ROC curves offer a better alternative. ROC is a plot of signal (True Positive Rate) against noise (False Positive Rate). The model performance is determined by looking at the area under the ROC curve (or AUC). The best possible AUC is 1 while the worst is 0.5 (the 45 degrees random line). Any value less than 0.5 means we can simply do the exact opposite of what the model recommends to get the value back above 0.5.

Check out [this website](https://towardsdatascience.com/receiver-operating-characteristic-curves-demystified-in-python-bd531a4364d0) for more info on ROC curves.

Let's compare this to the algorithm's performance on the training data:

In [ ]:
from sklearn.metrics import roc_curve, auc
y_pred = log_reg.predict(test_X)
fpr, tpr, _ = roc_curve(test_y, y_pred)
roc_auc = auc(fpr, tpr)
print(roc_auc)

In [ ]:
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

## Discuss: is our model a good model?

# 2. Multiple Logistic Regression

Multiple logistic regression models is the generalization of logistic regression to multiple predictors. That is, multiple logistic regression models the relationship between a binary outcome $Y$ that can be either 1 or 0 and multiple predictors $X_i$. 

The model is given by the equation

$$ ln \bigg(\frac{p(Y)}{1-p(Y)} \bigg) = c_0 + c_1 X_1 + c_2 X_2 + c_3 X_3 + \dots + c_n X_n + \epsilon$$

where $p(Y)$ is the probability that $Y=1, $$c_0$ is the regression coefficient associated with the probability that $Y=1$ when $X_i=0$ for all $i$, and $c_i$ is the regression coefficient that tells us how much the probability that $Y=1$ changes when the predictor $X_i$ is not zero. As before, we will set $\epsilon$ to zero, meaning we will ignore the effects of noise.

The best-fit curve provides a nonlinear model which allows us to make predictions about the probability that $Y=1$ or $Y=0$ given the values of $X_i$.

# 2.1 Multiple logistic regression for child's low birth weight using father's engagement and race as predictors
In this section we will build a multiple logistic regression model to predict the child's low birth weight. As a predictor, we will use both father engagement and the father's race (using one-hot encoding).

## Problem 5: 

Build a multiple logistic regression model to predict the child's low birth weight. As predictors use father engagement and the father's race (using one-hot encoding).

In [ ]:
# Create a DataFrame df1 containing "low birth weight", "father engagement" and "father race"


In [ ]:
# Use one-hot encoding to create a DataFrame df2 with "low birth weight", "father engagement" 
# and 4 binary columns representing the one-hot encoding of the "father race" categorical feature.


In [ ]:
# Split df2 into training and testing sets
# Also, don't forget to transform the sets from Pandas Series to numpy arrays


In [ ]:
# Construct and train a logistic regression model and print the model coefficients


In [ ]:
# Test your model with the test data using the classification accuracy and the ROC AUC.


In [ ]:
# Plot the ROC curve


## Discuss:

- Did the model performance improve by adding the father's race as a predictor?
- What other features could we add as predictors that might have a strong effect on model performance?